In [11]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
%reload_ext autoreload
%autoreload 2
from HelperFunctions import feature_standardize, label_encode_column, columns_of_type

from sklearn.model_selection import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
reg = RandomForestRegressor()
clf = RandomForestClassifier()

import pickle

In [12]:
# df=pd.read_csv('Sampled_Dataset_Balanced.csv')
total_df=pd.read_csv('down_sampled_df_v2.csv')
# temp_pre_df=pd.read_csv('pre_downsample_df.csv')
# pre_df=temp_pre_df.sample(50000)

In [13]:
# Defining the time windows:
timeWindows=[('2007-07','2007-09'),('2007-10','2007-12'),
             ('2008-01','2008-03'),('2008-04','2008-06'),
             ('2008-07','2008-09'),('2008-10','2008-12'),
             ('2009-01','2009-03'),('2009-04','2009-06'),
             ('2009-07','2009-09'),('2009-10','2009-12'),
             ('2010-01','2010-03'),('2010-04','2010-06'),
             ('2010-07','2010-09'),('2010-10','2010-12'),
             ('2011-01','2011-03'),('2011-04','2011-06'),
             ('2011-07','2011-09'),('2011-10','2011-12'),
             ('2012-01','2012-03'),('2012-04','2012-06'),
             ('2012-07','2012-09'),('2012-10','2012-12'),
             ('2013-01','2013-03'),('2013-04','2013-06'),
             ('2013-07','2013-09'),('2013-10','2013-12'),
             ('2014-01','2014-03'),('2014-04','2014-06'),
             ('2014-07','2014-09'),('2014-10','2014-12'),
             ('2015-01','2015-03'),('2015-04','2015-06'),
             ('2015-07','2015-09'),('2015-10','2015-12'),
             ('2016-01','2016-03'),('2016-04','2016-06'),
             ('2016-07','2016-09'),('2016-10','2016-12'),
             ('2017-01','2017-03'),('2017-04','2017-06'),
             ('2017-07','2017-09'),('2017-10','2017-12'),
             ('2018-01','2018-03'),('2018-04','2018-06'),
             ('2018-07','2018-09'),('2018-10','2018-12'),
            ]
total_df.issue_d=pd.to_datetime(total_df.issue_d)

In [14]:
file_clf = open("pickle_model_clf.pkl",'wb')
file_reg_def = open("pickle_model_reg_def.pkl",'wb')
file_reg_paid = open("pickle_model_reg_paid.pkl",'wb')

In [15]:
grade_weight_dict={'A':1,
            'B':1,
            'C':1,
            'D':1,
            'E':1,
            'F':1,
            'G':1}

In [16]:
for timeIncrement in timeWindows:
    print(timeIncrement)
    df=total_df.loc[(total_df.issue_d>=timeIncrement[0]) & (total_df.issue_d<=timeIncrement[1]),:]

    df.loc[:,'weight']=df.loc[:,'grade'].map(grade_weight_dict)
    grade_weight_array=df.loc[:,'weight'].values
    index = df.index
    grade_weight_frame = pd.DataFrame(grade_weight_array, index=index)

    drop_list=['id','sub_grade','issue_d','zip_code','loan_duration','loan_status','return_rate','weight','RANDOM']

    x_train_clf=df.drop(drop_list,axis=1)
    x_train_reg_paid=df.loc[df.loc[:,'loan_status']=='Fully Paid',:].drop(drop_list,axis=1)
    x_train_reg_default=df.loc[df.loc[:,'loan_status']=='Default',:].drop(drop_list,axis=1)
    y_train_clf=df.loc[:,'loan_status']
    y_train_reg_paid=df.loc[df.loc[:,'loan_status']=='Fully Paid',:].loc[:,'return_rate']
    y_train_reg_default=df.loc[df.loc[:,'loan_status']=='Default',:].loc[:,'return_rate']

    cat_list=columns_of_type(x_train_clf,'string')

    x_train_clf.reset_index(drop=True,inplace=True)
    x_train_clf=label_encode_column(x_train_clf,cat_list)
    x_train_reg_paid.reset_index(drop=True,inplace=True)
    x_train_reg_paid=label_encode_column(x_train_reg_paid,cat_list)
    x_train_reg_default.reset_index(drop=True,inplace=True)
    x_train_reg_default=label_encode_column(x_train_reg_default,cat_list)

    grid_para_forest = {
    'n_estimators': [10,20,30],
    'max_depth':[3,4,5],
    'max_features':['auto'],
    'min_samples_leaf':[1,2,3],
    'min_samples_split':[2,3,4]
    }
    ######################################################################
    ########### Regression Score ######################
    def grade_weighted_r2(y_true,y_pred,sample_weight):        

            weighted_r2=r2_score(y_true,y_pred,sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1))
            return weighted_r2    

    score_params = {"sample_weight": grade_weight_frame}

    grade_weighted_scorer_r2 = make_scorer(score_func=grade_weighted_r2,
                                        greater_is_better=True,
                                        needs_proba=False,
                                        needs_threshold=False,
                                        **score_params)
    ########### Default Regression ####################
    grid_search_forest_reg_default = GridSearchCV(estimator = RandomForestRegressor(), param_grid = grid_para_forest,
                                          scoring=grade_weighted_scorer_r2,
                                          cv = 5, n_jobs = -1,iid=False)
    grid_search_forest_reg_default.fit(x_train_reg_default,y_train_reg_default)
    print('default reg score:',grid_search_forest_reg_default.score(x_train_reg_default,y_train_reg_default))
    ########### Fully-Paid Regression #################
    grid_search_forest_reg_paid = GridSearchCV(estimator = RandomForestRegressor(), param_grid = grid_para_forest,
                                          scoring=grade_weighted_scorer_r2,
                                          cv = 5, n_jobs = -1,iid=False)
    grid_search_forest_reg_paid.fit(x_train_reg_paid,y_train_reg_paid)
    print('fully paid reg score:',grid_search_forest_reg_paid.score(x_train_reg_paid,y_train_reg_paid))
    ######################################################################   
    ########### Classification Score ###############
    def grade_weighted_accuracy(y_true,y_pred,sample_weight):
            weighted_acc=accuracy_score(y_true,y_pred,
                                        sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1),
                                        normalize=True)
            return weighted_acc 

    score_params = {"sample_weight": grade_weight_frame}

    grade_weighted_scorer_acc = make_scorer(score_func=grade_weighted_accuracy,
                                        greater_is_better=True,
                                        needs_proba=False,
                                        needs_threshold=False,
                                        **score_params)
    ########### Classification #####################
    grid_search_forest_clf = GridSearchCV(estimator=RandomForestClassifier(),\
                                          param_grid=grid_para_forest,\
                                          n_jobs=-1,\
                                          scoring=grade_weighted_scorer_acc,cv=5,\
                                          return_train_score=False,iid=False)
    grid_search_forest_clf.fit(x_train_clf,y_train_clf)
    print('clf score:',grid_search_forest_clf.score(x_train_clf,y_train_clf))
    
    pickle.dump(grid_search_forest_clf, file_clf)
    pickle.dump(grid_search_forest_reg_default, file_reg_def)    
    pickle.dump(grid_search_forest_reg_paid, file_reg_paid)

('2007-07', '2007-12')


/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


default reg score: 0.7263896606491176
fully paid reg score: 0.8456804778019821
clf score: 0.96
('2008-01', '2008-06')


/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


default reg score: 0.7228947208593632
fully paid reg score: 0.879492575576517
clf score: 0.9016393442622951
('2008-07', '2008-12')


...

default reg score: 0.06329157647632822
fully paid reg score: 0.7187016787705629
clf score: 0.6867771178493279
('2018-01', '2018-06')


/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


default reg score: 0.16949179290048877
fully paid reg score: 0.7306081230565469
clf score: 0.7008547008547008
('2018-07', '2018-12')


/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


default reg score: 0.37837603552344334
fully paid reg score: 0.6113149230851909
clf score: 0.8259860788863109


In [18]:
file_clf.close()
file_reg_def.close()
file_reg_paid.close()

In [10]:
my_x,my_y=data_transform(df.sample(1000),drop_list,'loan_status',grade_weight_dict)
# my_x,my_y=data_transform(df,drop_list,'return_rate',grade_weight_dict)

NameError: name 'data_transform' is not defined

In [ ]:
# Load from file
with open('pickle_model_clf.pkl', 'rb') as file:
    pickle_model = pickle.load(file)
    
# Calculate the accuracy score and predict target values
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(my_x)

In [ ]:
with open('pickle_model_reg_def.pkl', 'rb') as file:
    pickle_model = pickle.load(file)
    pickle_model2 = pickle.load(file)
    pickle_model3 = pickle.load(file)
    pickle_model4 = pickle.load(file)

# Calculate the accuracy score and predict target values
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(my_x)

In [ ]:
with open('pickle_model_reg_paid.pkl', 'rb') as file:
    pickle_model = pickle.load(file)
    pickle_model2 = pickle.load(file)
    pickle_model3 = pickle.load(file)
    pickle_model4 = pickle.load(file)

# Calculate the accuracy score and predict target values
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(my_x)

In [ ]:
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model2.predict(my_x)

In [ ]:
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model3.predict(my_x)

In [ ]:
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model4.predict(my_x)

In [ ]:
score = pickle_model.score(my_x, my_y)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model5.predict(my_x)

In [ ]:
pickle_model3